<a href="https://colab.research.google.com/github/ALXAVIER-DEV/Spark/blob/master/Fixa%C3%A7%C3%A3o_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Running Pyspark in Colab**

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 3.0.1 with hadoop 2.7 and Java 8. The tools installation can be carried out inside the Jupyter Notebook of the Colab. One important note is that if you are new in Spark, it is better to avoid Spark 2.4.0 version since some people have already complained about its compatibility issue with python. 
Follow the steps to install the dependencies:

In [56]:
!apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install pyspark

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done
Building dependency tree       
Reading state information... Done
40 packages can be upgraded. Run 

Now that you installed Spark and Java in Colab, it is time to set the environment path which enables you to run Pyspark in your Colab environment. Set the location of Java and Spark by running the following code:

In [57]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Run a local spark session to test your installation:

In [58]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [59]:
spark

# Reading a CSV from google drive

Utilizando o Google Colab, é possível importar os datasets diretamente do Google Drive, sem ter que realizar o upload manual dos mesmos para a instância colab manualmente

In [60]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
spark.read\
  .option("inferSchema", "true") \
  .option("header", "true") \
  .option("delimiter", ",") \
  .csv("/content/drive/My Drive/Spark_Treinamento/vgsales.csv") \
  .show()

+----+--------------------+--------+----+--------+------------+--------+--------+--------+-----------+------------+
|Rank|                Name|Platform|Year|   Genre|   Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|
+----+--------------------+--------+----+--------+------------+--------+--------+--------+-----------+------------+
| 259|           Asteroids|    2600|1980| Shooter|       Atari|     4.0|    0.26|     0.0|       0.05|        4.31|
| 545|     Missile Command|    2600|1980| Shooter|       Atari|    2.56|    0.17|     0.0|       0.03|        2.76|
|1768|             Kaboom!|    2600|1980|    Misc|  Activision|    1.07|    0.07|     0.0|       0.01|        1.15|
|1971|            Defender|    2600|1980|    Misc|       Atari|    0.99|    0.05|     0.0|       0.01|        1.05|
|2671|              Boxing|    2600|1980|Fighting|  Activision|    0.72|    0.04|     0.0|       0.01|        0.77|
|4027|          Ice Hockey|    2600|1980|  Sports|  Activision|    0.46|

In [ ]:
!pip install pandas
!pip install matplotlib

# Questão 1
Utilizando o dataset vgsales:

https://www.kaggle.com/gregorut/videogamesales/data.

Recupere a quantidade de vendas globais (Global_Sales) da plataforma PS4, agrupado por gênero (Genre) entre os anos 2013 a 2019:

Obs.: realizar duas transformações. A primeira utilizando SparkSQL e a segunda utilizando PySpark.

In [64]:
data_dir = '/content/drive/My Drive/Spark_Treinamento/vgsales.csv'
df = spark.read.load(data_dir, format= "csv", inferSchema="true", header="true")

In [75]:
def display_df(df, n=20):
    return df.limit(n).toPandas()

In [76]:
display_df(df.describe(), 10)


,summary,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,count,16324,16324,16324,16324,16324,16324,16324,16324,16324,16324,16324
1,mean,8291.508270031854,1942.0,2600.0,2006.4042514089683,None,None,0.26546373437883625,0.14758147512864267,0.07867250673853998,0.048334354324916624,0.5403277382994689
2,stddev,4792.04373419048,NaN,0.0,5.826744311455716,None,None,0.8216584668393672,0.5088085165507903,0.3115844508522971,0.18990174560162928,1.5658596703325407
3,min,1,'98 Koshien,2600,1980,Action,10TACLE Studios,0.0,0.0,0.0,0.0,0.01
4,max,16600,¡Shin Chan Flipa en colores!,XOne,2016,Strategy,responDESIGN,41.49,29.02,10.22,10.57,82.74


In [103]:
# SparkSQL

df.createGlobalTempView("vgsales")

spark.sql("""
    SELECT 
    SUM(Global_Sales) as Total_vendas,
    COUNT(Global_Sales),
    Genre
    FROM
    vgsales
    where
    Platform = 'PS4'
    AND Year BETWEEN 2013 and 2019
    GROUP BY Genre
    ORDER BY Total_vendas DESC
    """
).show()




+------------------+-------------------+------------+
|      Total_vendas|count(Global_Sales)|       Genre|
+------------------+-------------------+------------+
| 87.06000000000002|                122|      Action|
|             75.32|                 34|     Shooter|
| 50.07000000000001|                 43|      Sports|
| 25.73999999999999|                 46|Role-Playing|
|             11.53|                 17|      Racing|
|              8.04|                 17|    Fighting|
|7.3999999999999995|                 15|        Misc|
| 7.010000000000001|                 11|    Platform|
| 4.699999999999999|                 19|   Adventure|
|0.7699999999999999|                  5|  Simulation|
|              0.41|                  5|    Strategy|
|              0.02|                  1|      Puzzle|
+------------------+-------------------+------------+



In [114]:
# Pyspark
import pyspark.sql.functions as f
df2 = df.select("Platform", "Year", "Genre","Global_Sales")\
  .where("Platform ='PS4' AND Year BETWEEN 2013 and 2019" )\
  .groupBy("Genre")\
  .agg(
      
      f.sum("Global_Sales").alias("total_vendas"),
      f.count("Global_Sales").alias("contagem_vendas")
  )\
  .orderBy(f.desc("total_vendas"))

df2.show()  

+------------+------------------+---------------+
|       Genre|      total_vendas|contagem_vendas|
+------------+------------------+---------------+
|      Action| 87.06000000000002|            122|
|     Shooter|             75.32|             34|
|      Sports| 50.07000000000001|             43|
|Role-Playing| 25.73999999999999|             46|
|      Racing|             11.53|             17|
|    Fighting|              8.04|             17|
|        Misc|7.3999999999999995|             15|
|    Platform| 7.010000000000001|             11|
|   Adventure| 4.699999999999999|             19|
|  Simulation|0.7699999999999999|              5|
|    Strategy|              0.41|              5|
|      Puzzle|              0.02|              1|
+------------+------------------+---------------+



# Questão 2

Imagine que você foi contratado como engenheiro de dados para construir e manter um data lake de dados. Uma das fontes que você pretende realizar a ingestão se encontra em um banco de dados relacional. Essa fonte consiste de uma tabela de vendas de uma organização contendo as seguintes colunas:

- venda_id
- data
- tipo_produto
- valor
- usuario

Além disso, você precisa disponibilizar essa informação para um usuário final que utilizará um servidor SparkSQL, permitindo que ele rode consultas utilizando a engine de processamento distribuído do Spark. Essas consultas serão executadas por uma ferramenta de visualização como o PowerBI.

Foi constatado também que 95% das queries terão filtros por tipo_produto em sua composição.

Como você desenharia a arquitetura dessa solução? Quais os melhores componentes? Como seria o seu pipeline de dados?

Descreva utilizando um desenho. Pode mandar uma foto do desenho (a caneta) ou usar uma ferramenta web para desenhar a arquitura (draw.io). Favor enviar diretamente para o professor no Slack.

# Questão 3

Implemente um word count da frase abaixo utilizando as Low Level APIs:


In [108]:
text = "Apache Spark is a unified analytics engine for large-scale data processing. It provides high-level APIs in Java, Scala, Python and R, and an optimized engine that supports general execution graphs. It also supports a rich set of higher-level tools including Spark SQL for SQL and structured data processing, MLlib for machine learning, GraphX for graph processing, and Structured Streaming for incremental computation and stream processing."


In [109]:
word_list = text.split(" ")

In [112]:
sc = spark.sparkContext

rdd = sc.parallelize(word_list)\
  .map(lambda word: (word, 1)) \
  .reduceByKey(lambda x , y: + x +y)
rdd.take(10)  

[('Apache', 1),
 ('Spark', 2),
 ('is', 1),
 ('unified', 1),
 ('analytics', 1),
 ('engine', 2),
 ('It', 2),
 ('provides', 1),
 ('high-level', 1),
 ('APIs', 1)]

In [113]:
sc.parallelize(word_list).count()

65